# Qubit spectroscopy: CZ Ramsey measurement example

Similar to a $T_2^*$ Ramsey experiment, a CZ Ramsey experiment measures fringes resulting from induced Z rotations, which can result in non-unitary CZs. To rectify this non-unitarity, we determine the correction we need to apply to each qubit in the form of `RZ` rotations. If a CZ is perfectly unitary (or has been compensated for adequately with `RZ` pulses), a CZ Ramsey experiment should return 0 radians for each qubit. If, however, some correction is required, these angles will be non-zero. 

A sample QUIL program at one data point (specified by the equatorial Z rotation which maximizes excited state visibility when equal to the required `RZ` correction) between qubits 0 and 1 would look like 

    DECLARE ro BIT[1]
    DECLARE theta REAL[1]
    RX(pi/2) 0
    CZ 0 1
    RZ(theta) 0
    RX(pi/2) 0
    MEASURE 0 ro[0]
    
Since we can natively parameterize `RZ` rotations continuously on our hardware, we are able to make use of [Quil 2.0's parametric program compilation](https://pyquil.readthedocs.io/en/stable/migration3-declare.html?highlight=parametric%20program%20compilation#Parametric-compilation) using `DECLARE theta REAL[1]` and `RZ(theta) 0`. 

## setup

In [ ]:
from typing import Tuple

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from pyquil import Program
from pyquil.api import get_qc, QuantumComputer
from forest_benchmarking.qubit_spectroscopy import (generate_cz_phase_ramsey_experiment,
                                                    acquire_data_cz_phase_ramsey,
                                                    estimate_cz_phase_ramsey,
                                                    plot_cz_phase_estimate_over_data)

## measurement, analysis, and plotting

This method takes in a `pandas.DataFrame` object with all the CZ Ramsey data, including the phases and qubits measured, and plots the CZ Ramsey fringes of each qubit with respect to increasing applied contrast phase. It fits to a sinusoid, evaluates the phases at which excited state visibility is maximized for each qubit, $\theta_0$ and $\theta_1$, and plots a vertical line at these maximum excited state visibility points over the experimental data. 

In [ ]:
#qc = get_qc('Aspen-1-15Q-A')
#qc = get_qc('Aspen-3-14Q-A',as_qvm = True)
#qc = get_qc('Aspen-3-14Q-A')
qc = get_qc('2q-noisy-qvm') # will run on a QVM, but not meaningfully 
qubits = qc.qubits()
graph = qc.qubit_topology()
edges = list(graph.edges())
edges

In [ ]:
#qc.device.specs

**CZ ramesey experiments**

We can specify which qubits `edges`

In [ ]:
exp_cz = generate_cz_phase_ramsey_experiment(edges)

In [ ]:
exp_cz

**Acquire data**

Collect our Rabi raw data using `acquire_data_cz_phase_ramsey`.

In [ ]:
result = acquire_data_cz_phase_ramsey(qc,exp_cz)

In [ ]:
result

**Use the results to produce estimates of Rabi curve**

In [ ]:
estimate_cz_phase_ramsey(result)

**Plot**

In [ ]:
plot_cz_phase_estimate_over_data(result)